In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
vdf = pd.read_csv('fruitvegprices-2017_2022.csv')

In [4]:
vdf

,category,item,variety,date,price,unit
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg
3,fruit,apples,braeburn,2022-03-11,1.05,kg
4,fruit,apples,gala,2022-03-11,1.03,kg
...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem


## Ejercicios

### Ejercicio uno: realice un analisis de valores nulos

En caso de existir valores nulos elimine esos registros

In [104]:
for column in vdf.columns:
    print(column)
    print(np.sum(vdf[column].isna()))

category
0
item
0
variety
0
date
0
price
0
unit
0
year
0
month
0


### Ejercicio dos: Por categoria, cuantos distintos items hay y cuantos distintos variety hay?

El resultado debe de tener el siguiente formato de presentacion:

In [6]:
pd.DataFrame({
    "category" : ['cut_flowers','fruit']
    , "item_count" : [9,10]
    , "variety_Count" : [10,21]
})

,category,item_count,variety_Count
0,cut_flowers,9,10
1,fruit,10,21


Como nota en un mismo dataframe mostramos todo lo solicitado, usted debe de mostrar todas las categorias

In [105]:
x = vdf.loc[:,["category","item"]].drop_duplicates()
y = vdf.loc[:,["category", "variety"]].drop_duplicates()

In [106]:
x = x.groupby(["category"], as_index=False).agg({'item':'count'})
y = y.groupby(["category"], as_index=False).agg({'variety':'count'})

In [107]:
x["variety"] = y["variety"]

In [109]:
x.columns = ["category", "item count", "variety count"]

In [110]:
x

,category,item count,variety count
0,cut_flowers,9,9
1,fruit,10,21
2,pot_plants,3,2
3,vegetable,32,49


### Ejercicio tres: Para la combinacion item, variety muestre el ultimo valor registrado por año.

Ejemplo de como debe de mostrar su respuesta

In [111]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year" : [2022]
    , "last_price" :  [3.2]
})

,item,variety,year,last_price
0,Manzana,Gala,2022,3.2


In [112]:
vdf["date"] = pd.to_datetime(vdf["date"])
vdf["year"] = vdf["date"].dt.year
vdf["month"] = vdf["date"].dt.month

In [113]:
result_last_price = pd.DataFrame()

In [114]:
for year in vdf.loc[:, "year"].drop_duplicates():
    n = vdf.loc[vdf["year"]==year, ["item", "variety", "date", "price", "year"]].drop_duplicates()
    n = n.sort_values(["item", "variety", "date"], ascending=False)
    n["rank"] = n.groupby(["item", "variety"], as_index=False)["date"].rank(method='first', ascending=False)
    n = n.loc[n["rank"]==1, ["item", "variety", "year", "price"]]
    result_last_price = pd.concat([result_last_price,n])
result_last_price.columns = ["item", "variety", "year", "last_price"]

In [115]:
result_last_price

,item,variety,year,last_price
27,turnip,turnip,2022,0.63
31,tulips,tulips,2022,0.29
26,swede,swede,2022,0.24
14,spring_greens,prepacked,2022,1.04
30,rhubarb,outdoor,2022,3.07
...,...,...,...,...
9274,apples,egremont_russet,2017,0.86
9273,apples,coxs_orange_group,2017,0.78
9272,apples,bramleys_seedling,2017,0.75
9275,apples,braeburn,2017,0.71


### Ejercicio cuatro: Segun el ejercicio tres, que tanto aumento o disminuyo el precio de cada item-variety 

Ejemplo de como se debe de mostrar su respuesta

In [116]:
pd.DataFrame({
    "item" : ['Manzana','Manzana','Manzana']
    , "variety" : ['Gala','Gala','Gala']
    , "year" : [2020,2021,2022]
    , "last_price" :  [2.7,2.5,3.2]
    , "delta" : [0, (2.5-2.7), (3.2-2.5)]
})

,item,variety,year,last_price,delta
0,Manzana,Gala,2020,2.7,0.0
1,Manzana,Gala,2021,2.5,-0.2
2,Manzana,Gala,2022,3.2,0.7


Segun nuestro ejemplo dummy para 2021 estuvo 0.2 centavos mas barato y en 2022 estuvo 0.7 centavos caro

In [117]:
l = result_last_price.sort_values(["item", "variety", "year"], ascending=True).reset_index(drop=True)

In [118]:
l["lagPrice"] = l.groupby(["item", "variety"], as_index=False)["last_price"].shift(1)

In [119]:
l["delta"] = l["last_price"] - l["lagPrice"]

In [120]:
l["delta"] = l["delta"].fillna(0)

In [121]:
l = l.drop("lagPrice", axis=1)

In [122]:
l

,item,variety,year,last_price,delta
0,alstromeria,indoor,2017,0.27,0.00
1,alstromeria,indoor,2018,0.33,0.06
2,apples,braeburn,2017,0.71,0.00
3,apples,braeburn,2018,0.70,-0.01
4,apples,braeburn,2019,0.85,0.15
...,...,...,...,...,...
392,turnip,turnip,2021,1.28,0.11
393,turnip,turnip,2022,0.63,-0.65
394,watercress,pillow_packs,2017,10.82,0.00
395,watercress,pillow_packs,2018,7.35,-3.47


### Ejercicio cinco: Cual es precio promedio por item, variety

Ejemplo de como se debe de mostrar su respuesta

In [24]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year-month" : ['2021-01']   
    , "avg-price" : [0.5]
})

,item,variety,year-month,avg-price
0,Manzana,Gala,2021-01,0.5


In [123]:
result = pd.DataFrame()

In [124]:
for year in vdf.loc[:, "year"].drop_duplicates():
    for month in vdf.loc[vdf["year"] == year, "month"].drop_duplicates():
        m = vdf.loc[(vdf["year"] == year) & (vdf["month"] == month), ["item", "variety", "price"]]
        m["year-month"] = "%s-%s" % (year, month)
        m = m.groupby(["item", "variety", "year-month"], as_index=False).agg({'price':'mean'})
        result = pd.concat([result, m])
result.columns = ["item", "variety", "year-month", "avg-price"]

In [125]:
result

,item,variety,year-month,avg-price
0,apples,braeburn,2022-3,1.0600
1,apples,bramleys_seedling,2022-3,1.9700
2,apples,coxs_orange_group,2022-3,1.1950
3,apples,egremont_russet,2022-3,1.1500
4,apples,gala,2022-3,1.0450
...,...,...,...,...
55,tomatoes,plum,2017-11,0.7000
56,tomatoes,round,2017-11,0.8750
57,tomatoes,vine,2017-11,1.3500
58,turnip,turnip,2017-11,0.8175


### Ejercicio seis: muestre por category,item,variety que tanto % ha aumentado su precio

Ejemplo: si para las manzanas gala el primer precio que se registro fue 2.00 y el ultimo precio que se registro es 4.00 este aumento un 100%

In [28]:
pd.DataFrame({
    "category": ["fruta","fruta"]
    ,"item" : ['Manzana','Manzana']
    , "variety" : ['Gala','egremont_russet']
    , "percentage-change-between-first-and-last-price" : [200,150]       
})

,category,item,variety,percentage-change-between-first-and-last-price
0,fruta,Manzana,Gala,200
1,fruta,Manzana,egremont_russet,150


In [126]:
v = vdf.loc[:, ["category", "item", "variety", "price", "date"]].drop_duplicates()

In [127]:
v1 = v.sort_values(["category", "item", "variety", "date"], ascending=False)
v2 = v.sort_values(["category", "item", "variety", "date"], ascending=True)

In [128]:
v1["rank"] = v1.groupby(["category", "item", "variety"], as_index=False)["date"].rank(method='first', ascending=False)
v2["rank"] = v2.groupby(["category", "item", "variety"], as_index=False)["date"].rank(method='first', ascending=True)

In [129]:
v1 = v1.loc[v1["rank"] == 1, :].reset_index(drop=True)
v2 = v2.loc[v2["rank"] == 1, :].reset_index(drop=True)

In [130]:
v2 = v2.sort_values(["category", "item", "variety", "date"], ascending=False).reset_index(drop=True)

In [131]:
v1["percentage-change-between-first-and-last-price"] = ((v1["price"] - v2["price"])/v2["price"])*100

In [132]:
v = v1.drop(["price", "date", "rank"], axis=1)

In [133]:
v

,category,item,variety,percentage-change-between-first-and-last-price
0,vegetable,watercress,pillow_packs,26.119403
1,vegetable,turnip,turnip,-25.882353
2,vegetable,tomatoes,vine,-18.181818
3,vegetable,tomatoes,round,-41.237113
4,vegetable,tomatoes,plum,233.333333
...,...,...,...,...
80,cut_flowers,narcissus,indoor,16.666667
81,cut_flowers,lillies,oriental,-28.571429
82,cut_flowers,gladioli,all_varieties,40.909091
83,cut_flowers,chrysanthemum,indoor_spray,-22.727273


### Ejercicio siete: Segun el ejercicio seis por category cual es el item,variety con mayor aumento de crecimiento 

Segun el ejemplo dummy que se utilizo en el ejercicio seis para la category fruta se mostraria la manzana gala unicamente

In [134]:
t = v.sort_values(["category","percentage-change-between-first-and-last-price"], ascending=False)

In [135]:
t["rank"] = t.groupby("category", as_index=False)["percentage-change-between-first-and-last-price"].rank(method='first', ascending=False)

In [136]:
t = t.loc[t["rank"] == 1, ["category", "item", "variety", "percentage-change-between-first-and-last-price"]]

In [137]:
t

,category,item,variety,percentage-change-between-first-and-last-price
4,vegetable,tomatoes,plum,233.333333
53,pot_plants,cyclamen,13_cm,104.000000
66,fruit,blackberries,blackberries,275.000000
77,cut_flowers,stocks,stocks,57.500000
